<a href="https://colab.research.google.com/github/Harshit-Agarwal-007/Deep-Learning/blob/main/tuesdaymorning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'celeb-df-v2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3120670%2F5380830%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240921%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240921T183800Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6bfa9cd7f884f4dbddc4a2faf3f41ccfed35cffa85bfcf61e8ba2ce26a5b9beedf681d4083b9e193f118c9f8a3c2e21de2b944926ba43c16d2d89693866ddd1e52ceab03f6c7a971a2157d7c3877d8d30889bf873e8f46863cf51a81b5e36cd8eedb9c56010b9230f67e20b26569c891cbf12e9fb137e87115a2a506dbfed07fb055a3b8156551f5ace702822c9ee8715264acb719d087c9325b7b5af5d58ac4cdca52524c7a4449bc2aee7a1e02a89c9fe6aad253ec8fdd510f59c005dd52b555487dacbf53a0c5e2a1cce9eddf4a626f135a7f6fbb4a37a98ac52d1519556afea952580360c8566b289c0449c34fdb00620e6ba12d17fa9717aebbc0968a7e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[=============================                     ] 5883125760 bytes downloaded

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# Paths to real and fake videos
real_videos_path = '/kaggle/input/celeb-df-v2/Celeb-real'
fake_videos_path = '/kaggle/input/celeb-df-v2/Celeb-synthesis'

def extract_frames(video_path, num_frames=70, resize=(64, 64)):
    frames = []
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    step = max(1, total_frames // num_frames)  # Ensure step is at least 1
    for i in range(0, total_frames, step):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, resize)
            frames.append(frame)
    cap.release()
    return np.array(frames)

# Extract frames and labels
X, Y = [], []

for video_file in tqdm(os.listdir(real_videos_path)):
    video_path = os.path.join(real_videos_path, video_file)
    frames = extract_frames(video_path)
    X.append(frames)
    Y.append(1)  # Label for real videos

i = 0
for video_file in tqdm(os.listdir(fake_videos_path)):
    if i == 590:
        break
    i += 1
    video_path = os.path.join(fake_videos_path, video_file)
    frames = extract_frames(video_path)
    X.append(frames)
    Y.append(0)  # Label for fake videos

# Padding frames to have the same length
max_frames = max([len(x) for x in X])
X = pad_sequences(X, maxlen=max_frames, padding='post', truncating='post', dtype='float32')

Y = np.array(Y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Dropout, Flatten, Dense, LeakyReLU


# Define the 3D CNN model
model = Sequential([
    Conv3D(32, (3, 3, 3), input_shape=(X_train.shape[1], 64, 64, 3)),
    LeakyReLU(alpha=0.09),
    MaxPooling3D(pool_size=(2, 2, 2)),
    Dropout(0.2),
    Conv3D(64, (3, 3, 3)),
    LeakyReLU(alpha=0.09),
    MaxPooling3D(pool_size=(2, 2, 2)),
    Dropout(0.2),
    Flatten(),
    Dense(128),
    LeakyReLU(alpha=0.09),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    epochs=65,
                    validation_split=0.1,
                    batch_size=16,
                   callbacks=[early_stopping])

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.2f}")

In [ ]:
# Load your trained model
# model = ...  # Load your trained model here

# Path to the video you want to test
# test_video_path = '/kaggle/input/celeb-df-v2/Celeb-synthesis/id0_id26_0008.mp4'
videos = '/kaggle/input/celeb-df-v2/YouTube-real'
array = []
i=0
x=0
high = float(0)
low = float(1)
for video in tqdm(os.listdir(videos)) :
#     i+=1
#     if i==0 :
#         break
    test_video_path = os.path.join(videos, video)


# Extract frames from the test video
    test_frames = extract_frames(test_video_path)

# Ensure the frames are padded to the same length as in training
    test_frames = pad_sequences([test_frames], maxlen=max_frames, padding='post', truncating='post', dtype='float32')

# Predict using the trained model
    prediction = model.predict(test_frames)
    num = prediction[0][0]
    if high<num:
        high = num
    if low>num:
        low = num
#     print(num)
    array.append(num)
#     print("Prediction = " + str(prediction))
# Interpretation
    if prediction < 0.15:
        print("The video is predicted to be REAL.")

    else:
        print("The video is predicted to be FAKE.")
        print(x)
        x+=1

print(array)


In [ ]:
print(high)
print(low)

In [ ]:
# Load your trained model
# model = ...  # Load your trained model here

# Path to the video you want to test
# test_video_path = '/kaggle/input/celeb-df-v2/Celeb-synthesis/id0_id26_0008.mp4'
videos = '/kaggle/input/celeb-df-v2/Celeb-synthesis'
array = []
i=0
x=0
high = float(0)
low = float(1)
for video in tqdm(os.listdir(videos)) :
    i+=1
    if i==300 :
        break
    test_video_path = os.path.join(videos, video)


# Extract frames from the test video
    test_frames = extract_frames(test_video_path)

# Ensure the frames are padded to the same length as in training
    test_frames = pad_sequences([test_frames], maxlen=max_frames, padding='post', truncating='post', dtype='float32')

# Predict using the trained model
    prediction = model.predict(test_frames)
    num = prediction[0][0]
    if high<num:
        high = num
    if low>num:
        low = num
#     print(num)
    array.append(num)
#     print("Prediction = " + str(prediction))
# Interpretation
    if prediction < 0.15:
        print("The video is predicted to be REAL.")

    else:
        print("The video is predicted to be FAKE.")
        print(x)
        x+=1

print(array)


In [ ]:
print(array)
print(low)

In [ ]:
model.save('Latest_SIH.h5')